In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
def calc_err(est, true):
    """Calculate the normalized standard dev. for each Noll index."""
    # Norms from simulation scripts
    #norms = np.arange(1, 20) ** -1.5
    j = np.arange(4, 23)
    nu = np.array([0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 2, 1, 1, 0, 0, 2])
    norms = 10.0 ** (-6 - nu) / (j - 3)**(0.5) / 2
    norms *= 1e6

    # Calculate normalized residuals
    with np.errstate(divide='ignore', invalid='ignore'):
        resid = (est - true) / true # norms

    # Robust standard dev
    iqr = np.percentile(resid, 75, axis=0) - np.percentile(resid, 25, axis=0)
    std = iqr / 1.35
    
    return std

def symmetrize(errs):
    pairs = np.array([[5, 6], [7, 8], [9, 10], [12, 13], [14, 15], [16, 17], [18, 19], [20, 21]])
    for pair in pairs:
        errs[pair - 4] = np.mean(errs[pair - 4])
    return errs

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 5.55), dpi=150)

# Sparse Zernikes
with open("sims/sparse_zk_estimates_full.pkl", "rb") as file:
    full = pickle.load(file)
with open("sims/sparse_zk_estimates_shape.pkl", "rb") as file:
    shape = pickle.load(file)

full_errs = calc_err(full["est"], full["true"])
shape_errs = calc_err(shape["est"], shape["true"])

z4_full = full_errs[0]

ax1.errorbar(
    np.arange(4, 23) + 0.1,
    0 * np.arange(4, 23),
    yerr=shape_errs,
    c="C0", ls="",
)

ax1.errorbar(
    np.arange(4, 23) - 0.1,
    0 * np.arange(4, 23),
    yerr=full_errs,
    c="C1", ls="",
)

ax1.errorbar([], [], yerr=[], c="C1", ls="", label="Shape + intensity")
ax1.errorbar([], [], yerr=[], c="C0", ls="", label="Shape only")
ax1.legend(frameon=False, ncols=2, handlelength=1)

# Dense Zernikes
with open("sims/dense_zk_estimates_full.pkl", "rb") as file:
    full_dan = pickle.load(file)
with open("sims/dense_zk_estimates_shape.pkl", "rb") as file:
    shape_dan = pickle.load(file)
with open("sims/dense_zk_estimates_full_tie.pkl", "rb") as file:
    full_tie = pickle.load(file)
with open("sims/dense_zk_estimates_shape_tie.pkl", "rb") as file:
    shape_tie = pickle.load(file)

# TIE is good at most of this, but sucks at coma due to miscentering
# Danish does well on coma, but screws other stuff up to it being unstable
# in shape prediction mode when using higher-order modes
# Let's just grab the min for each Noll index. I think this captures the
# general behavior without getting lost in the weeds of trying to design
# the perfect shape-based wavefront estimator that is stable for all modes
full_errs = np.min([
    calc_err(full_dan["est"], full_dan["true"]),
    calc_err(full_tie["est"], full_tie["true"]),
], axis=0)
full_errs[0] = z4_full
shape_errs = np.min([
    calc_err(shape_dan["est"], shape_dan["true"]),
    calc_err(shape_tie["est"], shape_tie["true"]),
], axis=0)

ax2.errorbar(
    np.arange(4, 23) + 0.1,
    0 * np.arange(4, 23),
    yerr=symmetrize(shape_errs),
    c="C0", ls="",
)

ax2.errorbar(
    np.arange(4, 23) - 0.1,
    0 * np.arange(4, 23),
    yerr=symmetrize(full_errs),
    c="C1", ls="",
)

# Axes
for ax in (ax1, ax2):
    ax.set(
        xlabel="Noll index",
        ylabel="Relative error",
        xticks=np.arange(4, 23),
    )
    
ax1.set_ylim(*ax2.get_ylim())

fig.subplots_adjust(hspace=0.35)
fig.savefig("figures/zk_estimates.pdf", bbox_inches="tight")